# DeepLabCut Toolbox - Colab
https://github.com/AlexEMG/DeepLabCut

This notebook illustrates how to use the cloud to:
- create a training set
- train a network
- evaluate a network
- create simple quality check plots
- analyze novel videos!

###This notebook assumes you already have a project folder with labeled data! 

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

Nath\*, Mathis\* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019.


Paper: https://www.nature.com/articles/s41596-019-0176-0

Pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf


In [ ]:
# Link google drive
from google.colab import drive
drive.mount('/content/drive')

## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [ ]:
#(this will take a few minutes to install all the dependences!)
!pip install deeplabcut

**(Be sure to click "RESTART RUNTIME" is it is displayed above above before moving on !)**

In [ ]:
# Use TensorFlow 1.x:
%tensorflow_version 1.x

YOU WILL NEED TO EDIT THE PROJECT PATH **in the config.yaml file** TO BE SET TO YOUR GOOGLE DRIVE LINK!

Typically, this will be: /content/drive/My Drive/yourProjectFolderName


In [ ]:
#Setup your project variables:
# PLEASE EDIT THESE:
ProjectPath = '/content/drive/My Drive/DLC_analysis/  
ProjectFolderName = 'ephys-Berkowitz-2020-09-18'
VideoType = 'avi' 

#don't edit these:
videofile_path = [ProjectPath+'Videos/'] #Enter the list of videos or folder to analyze.
videofile_path

#dest list 
dest_path = videofile_path


In [ ]:
#GUIs don't work on the cloud, so label your data locally on your computer! This will suppress the GUI support
import os
os.environ["DLClight"]="True"

In [ ]:
import deeplabcut

In [ ]:
deeplabcut.__version__

In [ ]:
#This creates a path variable that links to your google drive copy
#No need to edit this, as you set it up before: 
path_config_file = ProjectPath+ProjectFolderName+'/config.yaml'
path_config_file

## Create a training dataset:
### You must do this step inside of Colab:
After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

## Start training:
This function trains the network for a specific shuffle of the training dataset. 

In [ ]:
#let's also change the display and save_iters just in case Colab takes away the GPU... 
#if that happens, you can reload from a saved point. Typically, you want to train to 200,000 + iterations.
#more info and there are more things you can set: https://github.com/AlexEMG/DeepLabCut/blob/master/docs/functionDetails.md#g-train-the-network

deeplabcut.train_network(path_config_file, shuffle=1, displayiters=10,saveiters=500)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.03M iterations). 
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

**When you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating:
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [ ]:
%matplotlib notebook
deeplabcut.evaluate_network(path_config_file)
path_config_file
# Here you want to see a low pixel error! Of course, it can only be as good as the labeler, 
#so be sure your labels are good! (And you have trained enough ;)

## There is an optional refinement step you can do outside of Colab:
- if your pixel errors are not low enough, please check out the protocol guide on how to refine your network!
- You will need to adjust the labels **outside of Colab!** We recommend coming back to train and analyze videos... 
- pplease see the repo and protocol instructions on how to refine your data!

## Start Analyzing videos: 
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

* dynamic set to true to maintain tracking around targets

In [ ]:
deeplabcut.analyze_videos(path_config_file,videofile_path, videotype=VideoType,save_as_csv=True,dynamic = (True,.1,90))

## Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
deeplabcut.plot_trajectories(path_config_file,videofile_path, videotype=VideoType)

Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4. 

## Create labeled video:
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path, videotype=VideoType)